<a href="https://colab.research.google.com/github/SavvyCode15/Sentiment-analysis-with-langchain/blob/main/Sentiment_Analysis_with_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ASPECT BASED SENTIMENT ANALYSIS

In [ ]:
pip -q install langchain  google.generativeai

In [ ]:
import os
os.environ['GOOGLE_API_KEY']="ENTER_API_KEY"

In [ ]:
from langchain.llms import GooglePalm
llm=GooglePalm()

In [ ]:
from langchain import LLMChain, PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)
chain=LLMChain(llm=llm, prompt=prompt)

question = "Who was the president in the year Justin Bieber was born?"
print(chain.run({"question": question}))




In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import FewShotPromptTemplate
from langchain.chains import SimpleSequentialChain

In [ ]:
# This is an LLMChain for ASpect EXtraction
examples=[{
    "review":''' Improve the communication updates to families. Improve consistency, with housekeeping, it was supposed to be weekly but now it is every two or three weeks.\
    There is no consistency in the staff either due to high turnoverrate. Improve the services in the dining room along with meal options.''',
    "aspects":'''Relevent aspects are communication on updates, housekeeping consistency, staff turnover, and dining services.'''
},
{
    "review":''' On paper they do , but my wife has not been brought to them, I have not had a meeting to set up a plan for her. No one wheels her to partake in acivities.''',
    "aspects":''' Relevent aspects are staff Attitude, care plan setup, Staff involvement in activities. '''
}
]

In [ ]:
# configure a formatter that will format the few shot examples into a string. This formatter should be a PromptTemplate.
prompt_template='''
Review:{review}
{aspects}
'''
example_prompt=PromptTemplate(input_variables=["review", "aspects"], template=prompt_template)

print(example_prompt.format(**examples[0] ))

In [ ]:
final_prompt=FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Review:{review}/n",
    input_variables=["review"],
    prefix="We are extracting the aspects from the review given by the residents of nursing homes. Take the review as input and extract the different aspects about the staff, food, memory building activites, management, cost of the nursing home. Then return the aspects as a list."

)
print(final_prompt.format(review=''' The community has popcorn days, church, birthday celebrations, holiday parties, therapy dogs and so much more. mymother iis very happy here, and she is kept active'''))

In [ ]:
aspect_extraction_chain=LLMChain(llm=llm, prompt=final_prompt, output_key='aspects')


In [ ]:
output=aspect_extraction_chain.predict(review="The community has popcorn days, church, birthday celebrations, holiday parties, therapy dogs and so much more. mymother iis very happy here, and she is kept active")

In [ ]:
output

Sentiment Chain

In [ ]:
#configure a formatter that will format the few shot examples into a string. this formatter should be a Prompt Template
prompt_template='''
Given below review and extracted aspects, Tell me about the sentiment of these aspects. e.g. 'Positive', Negative', 'Neutral' in a format like (aspect, sentiment).
Review:{review}
Aspects:{aspects}
[(Aspect1, Sentiment of Aspect1), (Aspect2, Sentiment of Aspect2),....]
'''
example_prompt2=PromptTemplate(input_variables=["review", "aspects"], template=prompt_template)

print(example_prompt2)

In [ ]:
aspect_sentiment_chain=LLMChain(llm=llm, prompt=example_prompt2, output_key="Aspects_with_sentiment")

Final Sequential Chain



In [ ]:
from langchain.chains.sequential import SequentialChain
overall_chain= SequentialChain(
    chains=[aspect_extraction_chain, aspect_sentiment_chain],
    input_variables=["review"],
    output_variables=["review", "aspects", "Aspects_with_sentiment"],
    verbose=True
)

In [ ]:
overall_chain({"review":"The staff here is really nice and cooperative. But I don't like their dinner menu"})